# <center>**Course Project 2** </center>

**Policy**

1. You may use your own or any software packages/module/codes to do this project. 
2. The project has to be done in groups of 2-4 persons each working together to complete all the tasks. The team can be the same as the Project 1 or re-formed.  
4. The report should be no more than 20 pages, including one page describing the contributions of each of the members in the group, figures, tables and codes. 
5. You are encouraged to do beyond the described tasks along with the central topic. Quality and innovative work will be greatly rewarded. 


Consider a penetration of a bullet onto a plate, as shown in the following figure. 

<div>
<br>
<img src="../../../images/Penetration.png" width="300"/>
<br>
[A bullet penetrated a circular plate]
</div>

Study the paper entitle "Real-time prediction of projectile penetration to laminates by training machine learning models with finite element solver as the trainer" given in the file holder of the course website. The dataset for this study was generated from a large number of FEM analyses for a real research project, and it is given also in the file folder. The Python code for the neural network is given there as Neural_network_FEA.py. Note that this neural network model is for predicting the entire time history of the velocity of the bullet, before and after the penetration. Before doing your tasks before, you should try to repeat the work on NN first using exactly the same dataset and codes.  

1. Establish a new NN model to predict only the residual velocity of the bullet, using the given dataset, aiming to outperform the one in the paper. Note that for the penetrated cases, the residual velocity will still be positive. For the not-penetrated cases, it is negative because the bullet is bounced back. 

2. Establish any other alternative model to repeat Task 1. 

3. Discuss on what type of ML model work best for this type of data.

You may use any existing 3rd party ML module or your own code to get this tasks done. 

Please submit both codes and reports online by the deadline. No extension will be given, because this is the final project. 